In [ ]:
import pandas as pd
import cv2
import os
import ast 
from concurrent.futures import ThreadPoolExecutor, as_completed


files_dir='/mnt/challenge_preprocessing/files'


def all_images_same_shape(path, img_range_str):
   
    global scan
    img_range = ast.literal_eval(img_range_str)
    start, end = img_range  
    
    path = path.replace('preprocessed', 'OriginalDatauncompressed')

    end=len(os.listdir(path))

    shapes = set()

    scan+=1
      
    for i in range(0, end):
        img_path = os.path.join(path, f"{i}.jpg")
        img = cv2.imread(img_path)
        if img is not None:
            shapes.add(img.shape)
        else:
            return False
    
    return len(shapes) == 1 


def process_row(row):
    path, img_range_str = row['Path'], row['Range']
    if all_images_same_shape(path, img_range_str):
        return row
    else:
        return None


    
def filter_df_concurrently(df):

    filtered_rows = []
    with ThreadPoolExecutor() as executor:

        futures = [executor.submit(process_row, row) for _, row in df.iterrows()]
        
        for future in as_completed(futures):
            result = future.result()
            if result is not None:
                filtered_rows.append(result)
    

    return pd.DataFrame(filtered_rows)



In [ ]:
df = pd.read_csv(os.path.join(files_dir,'la-challenge1_train_path_range_label.csv'))

scan=0
filtered_df = filter_df_concurrently(df)

shuffled_df = filtered_df.sample(frac=1).reset_index(drop=True)
print(shuffled_df)
print(df)
shuffled_df.to_csv(os.path.join(files_dir,'filtered-la-challenge1_train_path_range_label.csv'), index=False)

In [ ]:
df = pd.read_csv(os.path.join(files_dir,'la-challenge1_valid_path_range_label.csv'))

scan=0
filtered_df = filter_df_concurrently(df)

shuffled_df = filtered_df.sample(frac=1).reset_index(drop=True)
print(shuffled_df)
print(df)
shuffled_df.to_csv(os.path.join(files_dir,'filtered-la-challenge1_valid_path_range_label.csv'), index=False)